<a href="https://colab.research.google.com/github/oisincam/quantum_circuits/blob/main/stim_tableau_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stim~=1.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 15.3 MB/s eta 0:00:00


In [ ]:
import stim
import numpy as np

#Introduction

The aim of this notebook is to explain the basics of using stim's tableau simulator.

Tableau simluation is a means by which certain quantum circuits can be efficiently simulated on a classical computer. This is due to the fact that for a circuit containing only [Clifford Gates](https://en.wikipedia.org/wiki/Clifford_gate), updates to the state vectore can be tracked using the stabilizer of the state. The stabilizer of an $n$-qubit can be described with $2n^2$

##Basic Simulations

The stim circuit framework is great, but less customisable than the Tableau Simulator

This will let us directly and interactively simulate a circuit using the tableau simulation. Let us prepare the Bell state and measure it.

In [ ]:
s = stim.TableauSimulator()
s.h(0)
s.cnot(0,1)
s.measure(0)
s.measure(1)
s.current_measurement_record()

[False, False]

A super duper nice thing about the simulator is that we can define a circuit it and feed it to the current simulator state.


In [ ]:
s = stim.TableauSimulator()
s.h(0)

In [ ]:
circuit=stim.Circuit()
circuit.append('CX',[0,1])
circuit.append('M',[0])
circuit.append('M',[1])

In [ ]:
s.do(circuit)

In [ ]:
s.current_measurement_record()

[True, True]

Let's implement the 5 qubit code

In [ ]:
n=5 #number of data qubits

In [ ]:
stabilizer=['XZZXI','IXZZX','XIXZZ','ZXIXZ']#stabilizers
LogicalZ=['ZZZZZ']
LogicalX=['XXXXX']

#Suitable operators to fix the signs of the Fullstabilizer
#The ith operator anticommutes with Fullstabilizer i
Fixers=['IZIZZ','IIIIZ','ZZIZZ','IZIIZ','ZIIZX']

FullStabilizer=stabilizer+LogicalZ

In [ ]:
n=len(stabilizer[1])#number of logical qubits
t=len(stabilizer)#number of measurements= number of required ancillas
k=n-t

In [ ]:
#begin the simulation
s = stim.TableauSimulator()

In [ ]:
#create a new circuit to implement state preparation
#We must measure each stabilizer, then fix the signs at the end.
#We also have to measure the logical Z
circuit_StatePrepMeasure=stim.Circuit()

#Measure each FullStabilizer
for j in range(0,n):#for each FullStabilizer

  circuit_StatePrepMeasure.append('H', [n])#apply H to ancilla
  for i in range(0,n): #for each qubit in the stabilizer
    if FullStabilizer[j][i]!='I':
      op='C'+FullStabilizer[j][i]
      circuit_StatePrepMeasure.append(op,[n,i]) ##apply controlled operations based on stabilizer
  circuit_StatePrepMeasure.append('H', [n])##apply H to ancilla
  circuit_StatePrepMeasure.append('M', [n])##measure ancilla qubit
  circuit_StatePrepMeasure.append('R',n)#reset the ancilla to the 0 state
circuit_StatePrepMeasure.append('TICK')


circuit_StatePrepMeasure.append('TICK')
circuit_StatePrepMeasure.diagram()



/------------------------------------------------------------------------------------------------------------------\
q0: ---X---------------------------------------------X----------------------@----------------------@------------------------
       |                                             |                      |                      |
q1: ---|-@--------------------X----------------------|----------------------|-X--------------------|-@----------------------
       | |                    |                      |                      | |                    | |
q2: ---|-|-@------------------|-@--------------------|-X--------------------|-|--------------------|-|-@--------------------
       | | |                  | |                    | |                    | |                    | | |
q3: ---|-|-|-X----------------|-|-@------------------|-|-@------------------|-|-X------------------|-|-|-@------------------
       | | | |                | | |                  | | |                  | | |                  | | | |
q4: ---|-|-|-|----------------|-|-|-X----------------|-|-|-@----------------|-|-|-@----------------|-|-|-|-@----------------
       | | | |                | | | |                | | | |                | | | |                | | | | |
q5: -H-@-@-@-@-H-M:rec[0]-R-H-@-@-@-@-H-M:rec[1]-R-H-@-@-@-@-H-M:rec[2]-R-H-@-@-@-@-H-M:rec[3]-R-H-@-@-@-@-@-H-M:rec[4]-R---
     \------------------------------------------------------------------------------------------------------------------/

In [ ]:
s.do(circuit_StatePrepMeasure)

In [ ]:
s.current_measurement_record()

[True, True, False, True, False]

In [ ]:
#Now we want to apply the fixes, based on the current measurement record
circuit_StatePrepFix=stim.Circuit()
for i in range(0,n):
  if s.current_measurement_record()[i]==True:
    for qubit in range(0,n):
      circuit_StatePrepFix.append(Fixers[i][qubit],qubit)
circuit_StatePrepFix.diagram()


q0: -I-I-I-

q1: -Z-I-Z-

q2: -I-I-I-

q3: -Z-I-I-

q4: -Z-Z-Z-

In [ ]:
#Let's Apply the fix
s.do(circuit_StatePrepFix)

In [ ]:
#Let's measure everything again to check it works
s.do(circuit_StatePrepMeasure)


In [ ]:
s.current_measurement_record()


[True, True, False, True, False, False, False, False, False, False]

##Useful Functions

It will be nice to have functions that can generate a circuit for a specific purpose. For example, to measure a stabilizer using a fresh ancilla


First, we will make a timeline circuit which will be able to show the full timeline at the end of the simulation

In [ ]:
#given a stabilizer string of length n, measure with one ancilla
#This is not fault tolerant
#we assume n is the number of data qubits
def CircuitMaker_MeasurePauli(pauli):
  c=stim.Circuit()
  c.append('R',n)#reset the ancilla to the 0 state
  c.append('H', [n])#apply H to ancilla
  for i in range(0,n): #for each qubit in the stabilizer
    if pauli[i]!='I':
      op='C'+pauli[i]
      c.append(op,[n,i]) ##apply controlled operations based on stabilizer
  c.append('H', [n])##apply H to ancilla
  c.append('M', [n])##measure ancilla qubit
  c.append('R',n)#reset the ancilla to the 0 state
  return c


In [ ]:
(CircuitMaker_MeasurePauli('IXZZX')).diagram()

q0: --------------------------

q1: -----X--------------------
         |
q2: -----|-@------------------
         | |
q3: -----|-|-@----------------
         | | |
q4: -----|-|-|-X--------------
         | | | |
q5: -R-H-@-@-@-@-H-M:rec[0]-R-